## Solar eigenfrequencies and eigenfunctions

The individual mode functions for the spatial displacement, $\vec{\xi}_{q} \equiv \delta \vec{r}$, are eigenfunctions of a Sturm-Liouville problem defined through

\begin{equation}
    \mathcal{L}\cdot \vec{\xi}_{q}=\omega^2 \vec{\xi}_{q},
\end{equation}

where $\mathcal{L}$ is a differential operator and $q = \{n,l,m\}$ (see the paper for more details). Solving the Sturm-Liouvile problem leads to the eigenspectrum of acoustic standing waves in the solar interior. The diferential operator $\mathcal{L}$ depends on the equilibirum structure of the Sun through the density, sound speed, e.t.c. 
We decompose the eigenfunctions $\vec{\xi}_{q}$ into a **radial component**  ($\xi^r_{q}$), and **a horizontal component** ($\xi^h_{q}$) respectively. The radial component points along the propagation of the acoustic wave. 

We compute a calibrated solar model using the stellar evolution code MESA (see paper). Based on it, we solve the Sturm-Liouvile problem for adiabatic oscillations using the pulsation software GYRE. In the supporting folder we include the following files:


**1. GS98_OPAL_CALIBR.txt**: This is the equilibirum solar model in tabulated form (radius, density, pressure, e.t.c).


**2. summary_mesa.txt**: This is the file with the computed solar eigenfrequencies for degree $l = 1$ and a limited set of overtones $n$. 


**3. detail.l1.n+#.h5**: Tabulated eigenfunction for $l = 1$ and given overtone $n = #$. For example, the file with the first overtone reads detail.l1.n+1.h5. We provide the tabulated eigenfunctions for the first 5 overtones only.

All of these files need to be loaded to run the notebook. The relevant inlists to produce the equilibrium solar model and the pulsation data are also provided in the folder. The latter require the installation of MESA and GYRE. They can be downloaded at https://docs.mesastar.org/en/release-r22.11.1/index.html and https://gyre.readthedocs.io/en/stable/ respectively. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py # this is to read h5 files
from scipy.interpolate import interp1d 
from tabulate import tabulate 

# DEFINE the path where the folder with the files is located. 
# You will need to change this
file_path = '/Users/ippocratissaltas/Desktop/axions_gyre/'

###### SOLAR MODEL ######
# Note columns convection in file: (r, rho) = (10,1) 
solar_ref_file = file_path + 'GS98_OPAL_CALIBR' + '.txt' # Load the file with tabulated density, sound speed, etc. 
solar_ref = np.array(np.loadtxt(solar_ref_file)) # Import all solar model data
x_sun_model = sorted(solar_ref[:,10]) # Radial grid data - 1D list
rho_sun_model=  sorted(solar_ref[:,1],reverse=True) # Solar density data - 1D list 
###########################

###### PULSATION MODEL ######
pulse_model_file = file_path + 'detail.l1.n+1.h5' # File with the tabulated eigenfunction - 3D array 
                                                  # Here, degree is chosen as l = 1, and overtone n = 1.
pulse_model = h5py.File(pulse_model_file,'r') # Import all pulsation data
x_model = pulse_model['x'] # Radial grid data - 1D array. In principle different from solar model grid.
xi_r_model = pulse_model['xi_r']  # Radial eigenfunction data - 1D array
xi_h_model = pulse_model['xi_h']  # Horizontal eigenfunction data - 1D array 
###########################

######### PRINT EIGENFREQUENCIES ##################
summary_file =  file_path + 'summary_mesa.txt' # File with the pulsation eigenfrequencies 
summary = np.array(np.loadtxt(summary_file))    
freqs_table=[]
for i in range(0, len(summary)-1):     
    index_i = int(np.where(summary[:,2]== i)[0])
    freqi = (summary[index_i][3])*((1/86400)*10**6) # Frequencies are converted from cycles/day to Hz.
    #print(i, " | ", round(freqi,3))
    freqs_table.append([i, freqi])
     
freqs_table = np.array(freqs_table) # Prints out a table with the first overtones for visualisation purposes.
table = [['Overtone', 'Frequency [μHz]']]
for i in range(0,6):
    table.append(freqs_table[i,:].tolist())
print(tabulate(table,headers='firstrow', tablefmt='fancy_grid'))


╒════════════╤═══════════════════╕
│   Overtone │   Frequency [μHz] │
╞════════════╪═══════════════════╡
│          0 │           284.566 │
├────────────┼───────────────────┤
│          1 │           448.497 │
├────────────┼───────────────────┤
│          2 │           596.83  │
├────────────┼───────────────────┤
│          3 │           746.463 │
├────────────┼───────────────────┤
│          4 │           892.987 │
├────────────┼───────────────────┤
│          5 │          1038.94  │
╘════════════╧═══════════════════╛


## Computation of surface velocity field

Here we provide a very brief description of the equations needed to predict the surface velocity of a star due to the ULDM forcing term in the pulsation equations. The surface velocity amplitude of the star due to acoustic excitations is given by

\begin{equation}
V^{2}_{\rm{rms}} = |\vec{\xi}|^2_{r = R_{\odot}} \cdot \omega^{2}_{q} \cdot < |A(t)|^2 >.
\end{equation}

with $ |\vec{\xi}|^2_{r = R_{\odot}} \equiv \left( \xi_{r}^{2} + l(l+1)\xi_{h}^{2} \right)_{r = R_{\odot}}$. We will be denoting the solar eigenfrequency and ULDM frequency as $\omega_q$ and $\omega$ respectively. $\eta$ is the damping coefficient (units of frequency). We have further defined the time-averaged velocity amplitude as

\begin{equation}
    \label{eq:A_average}
    <|A_q|^2> = \Big( \frac{\mathcal{Q}^{(0)}_q}{4\omega_q^4I_q^2} \Big)\frac{\gamma^2 x^2 + 1}{(\gamma^2x^2 - 1)^2 + 4\eta^2/\omega_{q}^2},
\end{equation}

with the following definitions

\begin{equation}
    \int d^3\vec{x}\,\rho_0(\vec{x}')\vec{\xi}_{q}^*(\vec{x})\cdot\vec{\xi}_{q'}=I_q\delta_{qq'},
\end{equation}


\begin{equation}
\mathcal{Q}^{(0)}_q \equiv \frac{8\pi^2 \mathcal{W} G \rho \gamma v}{\sqrt{3}m},
\end{equation}
 

\begin{equation}
   \mathcal{W} \equiv  \int d^2r \, r^3 \,\mathfrak{f}_{nl}(r),
\end{equation}


\begin{equation}
    \mathfrak{f}_{nl}(r) \equiv \frac{d^2 (\rho_0\xi_{nl}^r)}{d r^2}+\frac{2}{r}\rho_0\xi_{nl}^r-\frac{l(l+1)}{r}\rho_0\xi^h_{nl}.
\end{equation}

$\rho_{0}(r)$ is the equilibirum stellar density loaded in the previous cell, and $\rho = \rm{constant}$ is the dark matter density. $\xi_{nl}^r(r), \xi_{nl}^h(r)$ are the radial and horizontal eigenfunctions respectively for given degree and overtone. $v$ is the velocity of the dark matter halo and $\omega=2m$ the mass of the dark matter particle. See the paper for more details.

In [2]:
####### This is the main function which produces the results ####### 

def velocity_eval(X, Y_r, Y_h, ω_p, η, σ, N_points = 100, ρ = 0.42, v = 7*10**-4, l = 1, plots = 'no'):
  
    ################################################
    # radius_min, radius_max = new radial grid to evaluate the interpolated eigenfunctions. Cannot be larger than original grid.
    # N_points = Number of points for the new grid for the interpolated eigenfunctions. 
    # X = 1D vector for original radial grid for the eigenfunction (Note the same with the grid of the solar profiles).
    # Y_r, Y_h = 1D vectors for radial, horizontal eigenfunctions tabulations.
    # ω_p = solar pulsation frequency [in μHz]. 
    # η = Friction coefficient [in mHz].
    # σ = Parametrises the frequency of the ULDM through  ω = ω_p + σ [in mHz].
    # Degree l is set to l = 1 by default.
    # ρ = ULDM denisty in GeV.
    # v = ULDM velocity as fraction of c. 
    # If plots = "yes" then the function outputs some plots for the density, eigenfunctions etc.
    ################################################
    
    Pi = np.pi # The constant π. 
    Rs = (6.9598)*10**10       # Solar radius in cm [cgs]
    Ms = (1.9892)*10**(33)     # Solar mass in g [cgs]
    G = (6.67428)*10**(-8)     # Newton's G in  (cm^3)*(g^-1)*(s^-2) [cgs]
    c = 3*10**10               # Speed of light in cm/sec [cgs]
    h = 4.1*(10**(-15))        # Units: ev/Hz 
    ρ_cgs = ρ*(10**(-24))      # GeV --> grams [cgs]
    v_cgs = v*c                # v is inputed as a fraction of c, so here I am converting to cm/s
    ω = ω_p*(1 + σ)            # ω is the frequency of the ULDM. σ parametrises the distance from exact resonance. 
    m = ω/2                    # mass of ULDM in Hz
    ε = 0.001                   # ε better not be changed. It is useful for part of the numerics.     
    
    def int(x,y): 
        y_max = y[1:]
        y_min = y[:-1]
        N = len(x)
        dx = Rs*(x[-1] - x[0])/(N-1)
        integral = (0.5*dx)*np.sum(y_max + y_min)
        return integral

    # 1D vectors with data & lists for tabulation
    x = X         # Spatial grid for the stellar radius
    xi_r = Y_r    # Radial eigenfunction 
    xi_h = Y_h    # Horizontal eigenfunction
    radius_min =  x[0]
    radius_max = x[-1]    
    data_eigenf_r = []
    data_eigenf_h = []
    data_Deigenf_r = []
    data_Deigenf_h = []
    data_rho = []
    data_Drho = []

   # Tabulates eigenfunctions in 2D arrays of [radius, eigenfunction].
    for i in range(0,len(x)):
        xi=x[i]
        xi_ri = Rs*xi_r[i][0]
        xi_hi= Rs*xi_h[i][0]
        data_eigenf_r.append([xi, xi_ri])
        data_eigenf_h.append([xi,  xi_hi])            
    data_eigenf_r = np.array(data_eigenf_r) # Radial eigenfunction data (list to array)
    data_eigenf_h = np.array(data_eigenf_h) # Horizontal eigenfunction data (list to array)
    eigenf_r_interp = interp1d(data_eigenf_r[:,0],data_eigenf_r[:,1], kind='quadratic') # Interpolate radial eigenfunction
    eigenf_h_interp = interp1d(data_eigenf_h[:,0],data_eigenf_h[:,1], kind='quadratic') # Interpolate horizontal eigenfunction
    
    # Computes the derivative of the solar density w.r.t radius.
    for i in range(0, len(x_sun_model) ):    
        x_suni = x_sun_model[i] # radius grid computed in MESA 
        x_suni_1 = x_sun_model[i-1]
        dx_sun = (x_suni - x_sun_model[i-1]) 
        rhoi = rho_sun_model[i]
        Drho = (rho_sun_model[i] - rho_sun_model[i-1])/dx_sun 
        if radius_min+ε < x_suni < radius_max -ε:
            data_rho.append([x_suni_1, rhoi])
            data_Drho.append([x_suni_1, Drho])
    data_rho = np.array(data_rho)
    data_Drho = np.array(data_Drho)
    rho_interp = interp1d(data_rho[:,0], data_rho[:,1], kind='linear') # check interpolation 
    Drho_interp = interp1d(data_Drho[:,0], data_Drho[:,1], kind='cubic') # derivative of solar density
    
    # Defines a new radial grid. 
    x_new = np.linspace(radius_min + 2*ε, radius_max - 2*ε, num=N_points, endpoint=True) # new grid for interpolation.     
         
    # Computes the derivative of the eigenfunctions. It uses the interpolated eigenfunctions.    
    for i in range(0,len(x_new)):
        xi = x_new[i] #x[i] 
        xi_1 = x_new[i-1] #x_new[i-1] #x[i-1]
        dx = (xi - xi_1) #(x[i] - x[i-1])
        if radius_min + ε < xi < radius_max - ε:
            Dfr   = (eigenf_r_interp(xi) - eigenf_r_interp(xi_1))/dx 
            Dfh   = (eigenf_h_interp(xi) - eigenf_h_interp(xi_1))/dx        
            data_Deigenf_r.append([xi_1, Dfr])
            data_Deigenf_h.append([xi_1, Dfh])
    data_Deigenf_r = np.array(data_Deigenf_r)
    data_Deigenf_h = np.array(data_Deigenf_h)
    Deigenf_r_interp = interp1d(data_Deigenf_r[:,0], data_Deigenf_r[:,1], kind='quadratic')
    Deigenf_h_interp = interp1d(data_Deigenf_h[:,0], data_Deigenf_h[:,1], kind='quadratic')
    
    # Defines the quantity f_nl.
    def fnl(r):
        return (1/Rs)*(eigenf_r_interp(r)*Drho_interp(r) + rho_interp(r)*Deigenf_r_interp(r) + (2/r)*rho_interp(r)*eigenf_r_interp(r) - (2/r)*rho_interp(r)*eigenf_h_interp(r))

    # Computes quantity W_nl(r) for degree l = 1. Remember: Wnl = integral(fnl*r^3) 
    Wnl_values = (Rs**3)*(x_new**3)*fnl(x_new) 
    Wnl = 0 
    dx_int = Rs*(x_new[-1] - x_new[0])/N_points
    Wnl = int(x_new, Wnl_values)
        
    def ksi_squared(r):
        return ( eigenf_r_interp(r)**2 + l*(l+1)*eigenf_h_interp(r)**2 ) 
        
    ksi_squared_surf = ksi_squared(x_new[-1]) #Normalisation of ksi_squared(r) at r = R_s.
    
    def integrand_I(r): # Integrand of moment of inertia. 
        return (Rs**2)*(r**2)*rho_interp(r)*ksi_squared(r)
    
    def I(r): # moment of inertia. I = integral(integrand_I). 
        I_values = integrand_I(r)
        I0 = int(x_new, I_values)
        return I0
       #  I0 = 0
       #  dx_int = Rs*(x_new[-1] - x_new[0])/N_points
       # for i in range(0,len(I_values)):
       #     I0 = I0 + I_values[i]*dx_int
       #     I0= I0
   
    ωfrac = 1 + σ # ωfrac =  ω/ω_p. 
    I_p = I(x_new) # evaluate moment of inertia I (function of I defined above.)
    
    c0 = ((16)*(Pi**4)/3)*( 1/( ((10**(-6)*ω_p)**4)*((10**(-6)*m)**2) ) )*((Wnl/I_p)*G*ρ_cgs*v_cgs)**2  
    A_squared_average = c0*( (ωfrac)**2 + 1 )/( (ωfrac**2 - 1)**2  + 4*(η/ω_p)**2 ) # < |A|^2 >
    
    # Velocity RMS squared
    v_rms_squared = (ksi_squared_surf)*A_squared_average*(10**(-6)*ω_p)**2
    
    # Velocity RMS 
    v_rms = np.abs(v_rms_squared)**(0.5) 
        
    ##### Plots the interpolated functions 
    if plots == 'yes':
        #plt.plot(data_eigenf_r[:,0], data_eigenf_r[:,0], 'o', x_new, eigenf_r_interp(x_new), '-')
        plt.plot(data_eigenf_r[:,0], data_eigenf_r[:,1])
        #plt.plot(x_new, x_new*(rho_interp(x_new)**0.5)*eigenf_r_interp(x_new), '-')
        plt.xlabel( 'r/R')
        plt.ylabel('Eigenf. y^radial [cm]')
        plt.xlim([0., 0.99])
        plt.show()
        #plt.plot(data_eigenf_h[:,0],data_eigenf_h[:,1], 'o', x_new, eigenf_h_interp(x_new), '-')
        plt.plot(data_eigenf_h[:,0],data_eigenf_h[:,1], '-')
        plt.xlabel( 'r/R')
        plt.ylabel('Eigenf. y^horiz. [cm]')
        plt.xlim([0., 0.99])
        plt.show()
        plt.plot(x_new, rho_interp(x_new), 'o', x_new, rho_interp(x_new), '-')
        plt.xlabel( 'r/R')
        plt.ylabel('solar density ρ [g/cm^3]')
        plt.show()
        plt.plot(data_Drho[:,0], data_Drho[:,1],'o', x_new, Drho_interp(x_new), '-')
        plt.xlabel( 'r/R')
        plt.ylabel('dρ/dr')
        plt.show()
        plt.plot(x_new, fnl(x_new))
        plt.xlabel( 'r/R')
        plt.ylabel('f_nl')
        plt.show()
        
     ###### Prints out the values of some useful quantities
        print('η(μHz) = ', η, ' | ', 'f (μHz) = ', ω_p)
        print(' ')
        print('(ξ*W_nl)/I_p =  ', (ksi_squared_surf**0.5)*Wnl/I_p )
        print(' ')
        print('< |A|^2 >  = ', A_squared_average)
        print(' ')
        print('v_rms = ', v_rms)
   
    return [(ksi_squared_surf**0.5)*(Wnl/I_p), v_rms]

In [3]:
# Evaluate the velocity RMS for representative values
η0 = 2*10**(-12)*10**6 # in μHz
freq0 = 300 # in μHz
σ0 = 0.0000 # ω = ω_p + σ : difference between ULDM and solar frequency
velocity_eval(x_model, xi_r_model, xi_h_model, freq0, η0, σ0, plots='no')

[0.6167967612542606, 1.950643697205863e-08]